In [1]:
import pandas as pd
import numpy as np
import os
from cassandra.cluster import Cluster
from datetime import datetime
import plotly.graph_objects as go


Connect cassandra

In [2]:
cluster = Cluster(['127.0.0.1']) 
session = cluster.connect()

In [3]:
# ดึง keyspace ทั้งหมด
rows = session.execute("SELECT keyspace_name FROM system_schema.keyspaces;")

# แสดงผลลัพธ์
print("Available Keyspaces:")
for row in rows:
    print("-", row.keyspace_name)

Available Keyspaces:
- system_auth
- system_schema
- system_distributed
- my_keyspace
- system
- settrade_api
- system_traces
- data_quran


Connenct Setrade API

In [4]:
import settrade_v2
from settrade_v2 import Investor

investor = Investor(
                app_id="rirpANWTJCOHVgnn",                                 
                app_secret="P7iCFYB6JX0cdLXogMwU0J2yNsrpne/nzX2pFy6kiA8=", 
                broker_id="SANDBOX",
                app_code="SANDBOX",
                is_auto_queue = False)

deri = investor.Derivatives(account_no="wanneeba-D")            

account_info = deri.get_account_info()

account_info                                              

{'creditLine': 2000000000.0,
 'excessEquity': 2000000000.0,
 'cashBalance': 2000000000.0,
 'equity': 2000000000.0,
 'totalMR': 0.0,
 'totalMM': 0.0,
 'totalFM': 0.0,
 'callForceFlag': 'No',
 'callForceMargin': 0.0,
 'liquidationValue': 2000000000.0,
 'depositWithdrawal': 0.0,
 'callForceMarginMM': 0.0,
 'initialMargin': 0.0,
 'closingMethod': 'Auto Net'}

In [5]:
#GetQuoteSymbol
mkt_data = investor.MarketData()
res = mkt_data.get_quote_symbol("AOT")
print(res)

{'instrumentType': 'STOCK', 'symbol': 'AOT', 'high': None, 'low': None, 'last': None, 'average': None, 'change': None, 'percentChange': None, 'totalVolume': 0, 'totalBuyVolume': 0, 'totalSellVolume': 4400, 'totalNoSideVolume': 9700, 'status': '', 'marketStatus': 'Close', 'securityType': 'CS', 'eps': 1.34277, 'pe': 38.73, 'pbv': 5.98, 'percentYield': 1.5199999809265137, 'maturityDate': None, 'exercisePrice': None, 'underlying': None, 'underlyingPrice': None, 'intrinsicValue': None, 'theoretical': None, 'moneyness': None, 'lastTradingDate': None, 'toLastTrade': None, 'exerciseRatio': None, 'impliedVolatility': None, 'exchange': None, 'aumSize': None, 'inav': None}


In [9]:
#Get Candlestick
market = investor.MarketData()
res = market.get_candlestick(
symbol="BBL",
interval="1d",
normalized=True,)
res

{'lastSequence': 543096,
 'time': [1677171600,
  1677430800,
  1677517200,
  1677603600,
  1677690000,
  1677776400,
  1678122000,
  1678208400,
  1678294800,
  1678381200,
  1678640400,
  1678726800,
  1678813200,
  1678899600,
  1678986000,
  1679245200,
  1679331600,
  1679418000,
  1679504400,
  1679590800,
  1679850000,
  1679936400,
  1680022800,
  1680109200,
  1680195600,
  1680454800,
  1680541200,
  1680627600,
  1680800400,
  1681059600,
  1681146000,
  1681232400,
  1681664400,
  1681750800,
  1681837200,
  1681923600,
  1682010000,
  1682269200,
  1682355600,
  1682442000,
  1682528400,
  1682614800,
  1682960400,
  1683046800,
  1683478800,
  1683565200,
  1683651600,
  1683738000,
  1683824400,
  1684083600,
  1684170000,
  1684256400,
  1684342800,
  1684429200,
  1684688400,
  1684774800,
  1684861200,
  1684947600,
  1685034000,
  1685293200,
  1685379600,
  1685466000,
  1685552400,
  1685638800,
  1685984400,
  1686070800,
  1686157200,
  1686243600,
  1686502800,
 

In [7]:
#Get Candlestick
market = investor.MarketData()
res = market.get_candlestick(
symbol="AOT",
interval="1d",
limit=1,
normalized=True,)
res

{'lastSequence': 349695,
 'time': [1743094800],
 'open': [88.0],
 'high': [88.0],
 'low': [68.0],
 'close': [88.0],
 'volume': [14100],
 'value': [1141225.0]}

In [10]:
# 👀 ตรวจสอบก่อนว่า res เป็น dict และมี key 'data' หรือไม่
if isinstance(res, dict) and "data" in res:
    raw_data = res["data"]
else:
    raw_data = res  # กรณี res เป็น list เลย
df = pd.DataFrame(raw_data)
df

,lastSequence,time,open,high,low,close,volume,value
0,543096,1677171600,163.5,165.0,163.0,164.0,8157533,1.336954e+09
1,543096,1677430800,163.5,164.0,161.5,162.5,4996492,8.119072e+08
2,543096,1677517200,162.5,165.0,162.0,163.5,6579318,1.074026e+09
3,543096,1677603600,165.0,165.0,163.0,163.0,4190779,6.852074e+08
4,543096,1677690000,163.0,163.5,157.5,158.5,16460035,2.624295e+09
...,...,...,...,...,...,...,...,...
495,543096,1742403600,148.0,149.0,148.0,148.5,647000,9.608135e+07
496,543096,1742490000,156.0,156.0,150.5,152.5,1271150,1.930659e+08
497,543096,1742749200,151.0,161.0,150.0,161.0,169600,2.571870e+07
498,543096,1742835600,160.5,161.0,130.0,148.0,1559120,2.458160e+08


In [11]:
df.columns

Index(['lastSequence', 'time', 'open', 'high', 'low', 'close', 'volume',
       'value'],
      dtype='object')

In [12]:
# แปลงวันที่ให้เป็น datetime object
df["time"] = pd.to_datetime(df["time"])
df.set_index("time", inplace=True)

# วาดกราฟ
fig = go.Figure()

fig.add_trace(go.Candlestick(
    x=df.index,
    open=df["open"],
    high=df["high"],
    low=df["low"],
    close=df["close"],
    name="Candlestick"
))

fig.add_trace(go.Bar(
    x=df.index,
    y=df["volume"],
    name="Volume",
    yaxis='y2',
    marker_color='rgba(150,150,150,0.3)'
))

fig.update_layout(
    title="กราฟแท่งเทียนหุ้น (จาก Settrade API)",
    xaxis_rangeslider_visible=False,
    yaxis=dict(title="ราคา"),
    yaxis2=dict(title="Volume", overlaying="y", side="right", showgrid=False),
    height=600,
)

fig.show()
